In [1]:
import numpy as np

from my_stuff import MufexKeys
from quantfreedom.enums import CandleProcessingType
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.strategies.strategy import Strategy
from quantfreedom.base import backtest_df_only
from quantfreedom.enums import *
from quantfreedom.helper_funcs import create_os_cart_product_nb

mufex = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
)
symbol = "BTCUSDT"
%load_ext autoreload
%autoreload 2

In [2]:
candles = mufex.get_candles_df(symbol=symbol, timeframe='5m', candles_to_dl=600)

It took 00 mins and 01 seconds to download the candles


In [3]:
mufex.set_exchange_settings(
    symbol=symbol,
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings()
order_settings_arrays = OrderSettingsArrays(
    risk_account_pct_size=np.array([1.0]) / 100,
    sl_based_on_add_pct=np.array([0.01, 0.02]) / 100,
    sl_based_on_lookback=np.array([50]),
    risk_reward=np.array([2.0, 5.0]),
    leverage_type=np.array([LeverageStrategyType.Dynamic]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    increase_position_type=np.array([IncreasePositionType.RiskPctAccountEntrySize]),
    stop_loss_type=np.array([StopLossStrategyType.SLBasedOnCandleBody]),
    take_profit_type=np.array([TakeProfitStrategyType.RiskReward]),
    max_equity_risk_pct=np.array([3]) / 100,
    long_or_short=np.array([LongOrShortType.Long]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct_from_candle_body=np.array([0.0]) / 100,
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.Nothing]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.High]),
    trail_sl_when_pct_from_candle_body=np.array([3.0]) / 100,
    trail_sl_by_pct=np.array([1.0]) / 100,
    static_leverage=np.array([0.0]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
    num_candles=np.array([50,100,200]),
)
os_cart_arrays = create_os_cart_product_nb(
    order_settings_arrays=order_settings_arrays,
)

In [4]:
strategy = Strategy(
    candle_processing_mode=CandleProcessingType.CandleBacktest,
    candles=candles,
)
backtest_df_only(
    starting_equity=1000.0,
    backtest_settings=backtest_settings,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=mufex.exchange_settings,
    candles=candles,
    strategy=strategy,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 12
Total combinations of settings to test: 72

Total candles: 600
Total candles to test: 43,200


,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
0,3,2,22.0,-3.924701,22.73,-0.134857,-37.7648,962.235144
1,0,2,22.0,-4.013204,22.73,-0.142700,-38.5836,961.416401
2,1,5,19.0,-3.737774,21.05,-0.164279,-36.0311,963.969017
3,2,5,19.0,-3.737774,21.05,-0.164279,-36.0311,963.969017
4,4,5,19.0,-3.737774,21.05,-0.164279,-36.0311,963.969017
...,...,...,...,...,...,...,...,...
67,1,9,21.0,-22.578764,19.05,-0.828458,-184.1979,815.801995
68,2,9,21.0,-22.578764,19.05,-0.828458,-184.1979,815.801995
69,4,9,21.0,-22.578764,19.05,-0.828458,-184.1979,815.801995
70,5,9,21.0,-22.578764,19.05,-0.828458,-184.1979,815.801995
